# INFO6105: Assignment 4 - Making your own Comic Book!

## The notebook follows the given order to create a comic book:

1. Import all required libraries
2. Resize Images
3. Add Border
4. Transforming the images into cartoon-type images and adding dailogues
3. Horizontally stitch the images
4. Vertically stitch the horizontal strips to form single images
5. Join these single final page images into a PDF

In [11]:
import os
import numpy as np
from PIL import Image ,ImageOps, ImageFont, ImageDraw
import pandas as pd
import cv2
from IPython.display import display

from fpdf import FPDF


In [12]:
#Change to your local path to Avoid error
path = '/Users/mrunalbhalerao/Desktop/ Data Science/HW/Glass Onion - Knives out - Part 2/'

In [13]:
fullpath=path+"/images-raw/"
dirs = os.listdir(fullpath)

#resize the images
def resize():
        for item in dirs: 
            if os.path.isfile(fullpath+item):
                if(item!='.DS_Store'):                                       
                    im = Image.open(fullpath+item)
                    f, e = os.path.splitext(fullpath+item)
                    imResize = im.resize((750,450), Image.ANTIALIAS)
                    imResize.save(f + '.jpg', quality=90)

In [14]:
def imgcompress_mem(path_in,k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # set the ratio of resized image
    width = int((img.shape[1])/2)
    height = int((img.shape[0])/2)

    # resize the image by resize() function of openCV library
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

In [15]:
def cartoonizebl_mem(path_in, k, blur, line):
    
    imgc = imgcompress_mem(path_in,k)
    #imgc_pil = cv2.cvtColor(imgc, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(imgc_pil))

    line_size = line
    blur_value = blur
    #imgc = cv2.imread(path_out, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(bigedges_pil))

    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

In [16]:
TINT_COLOR = (0, 0, 0)  # Black
OPACITY = int(255 * 0.75)
FONT = ImageFont.truetype("SourceSansPro.ttf", 24) # Font
IMG_BASE_WIDTH = 600
IMG_NUMBERS = 47

def convert_from_cv2_to_image(img: np.ndarray) -> Image:
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #return Image.fromarray(img)

def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    #return np.asarray(img)

In [17]:
def cartoonizeblt_mem_nb(path_in, k, blur, line, text, nlines=1, font='English'):
    
    imgc = imgcompress_mem(path_in, k)

    line_size = line
    blur_value = blur
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB

    toon = cv2.bitwise_and(imgc, imgc, mask=bigedges)
    if 0 == len(text):
        return toon
    
    myfont = (
        ImageFont.truetype("SourceSansPro.ttf", 
            24 if k == 16 else 18 if k == 14 else 18 if k == 12 else 20 if k == 8 else 82) if font=='English'
        else
            ImageFont.truetype("Aparajita.ttf", 
                24 if k == 16 else 18 if k == 14 else 18 if k == 12 else 20 if k == 8 else 82) if font=='Hindi'
        else
            ImageFont.truetype(font + ".ttf", 24 if k == 16 else 18 if k == 14 else 18 if k == 12 else 20 if k == 8 else 82)
    )


    cblimg_pil = Image.fromarray(cv2.cvtColor(toon, cv2.COLOR_BGR2RGBA))

    overlay = Image.new('RGBA', cblimg_pil.size, TINT_COLOR+(1,))
    draw = ImageDraw.Draw(overlay)
    #_, h = FONT.getsize(text)
    _, h = myfont.getsize(text)
    num_lines = nlines
    n=50
    if(font=='Hindi'):
        n=30
    #Checking the text length and deciding the number of lines
    if(len(text)>(n*2)):
        num_lines =3
        chunks = [text[i:i+n] for i in range(0, len(text), n)]
    elif(len(text)>n):
        num_lines =2
        chunks = [text[i:i+n] for i in range(0, len(text), n)]
        
    if(num_lines==1):
        x, y = 0, cblimg_pil.height - (num_lines)*h-10
        draw.rectangle((x, y, x + cblimg_pil.width, y + (num_lines)*h+10), fill=TINT_COLOR+(OPACITY,))
        if k == 1:
            draw.text((x+10, y), text, fill=(248,248,248), font=myfont) #, stroke_width=1)
        elif k < 8:
            draw.text((x+10, y), text, fill=(248,248,248), font=myfont)
        else:
            draw.text((x+10, y), text, fill=(248,248,248), font=myfont) #, stroke_width=1)
   
    #iterating and printing each line
    else:
        x, y = 0, cblimg_pil.height - (num_lines)*h-10
        draw.rectangle((x, y, x + cblimg_pil.width, y + (num_lines)*h+10), fill=TINT_COLOR+(OPACITY,))
        for i in range(0,len(chunks)):      
            if k == 1:
                draw.text((x+10, y), chunks[i], fill=(248,248,248), font=myfont) #, stroke_width=1)
            elif k < 8:
                draw.text((x+10, y), chunks[i], fill=(248,248,248), font=myfont)
            else:
                draw.text((x+10, y), chunks[i], fill=(248,248,248), font=myfont) #, stroke_width=1)
            y=y+15

    cblimg_pil = Image.alpha_composite(cblimg_pil, overlay)
    cblimg_pil = cblimg_pil.convert("RGB")
    
    return convert_from_image_to_cv2(cblimg_pil)


In [18]:
#Adding Borders
def borderImages(output_path):
    for filename in (dirs):
        if ('jpeg') in filename.lower():
            ImageOps.expand(Image.open(fullpath+filename), border=(10,10),fill='white').save(output_path+filename)
            ImageOps.expand(Image.open(output_path+filename), border=(10,10),fill='black').save(output_path+filename)

In [19]:
borderedImgPath =  path+"BorderedImage/"

In [20]:
#load the dialogues from the excel sheet
dialogues = pd.read_excel('GK - Scenes.xlsx')
dialogues #contains dialogues in english and the secondary language

,English,Hindi
0,.,.
1,.,.
2,"Andi: Helen's idea is great, let's tweak it a ...","एंडी: हेलेन का विचार महान है, चलो इसे थोड़ा ट्..."
3,"Inauguration of ""Paws Play""","""पंजे खेल"" का उद्घाटन"
4,.,.
...,...,...
66,Detective Blanc finally got to the bottom of t...,डिटेक्टिव ब्लैंक आखिरकार हत्या के मामले में सब...
67,.,।
68,Miles gets a message,माइल्स को एक संदेश मिलता है
69,Miles is feeling upset and angry.,माइल्स परेशान और गुस्से में महसूस कर रहे हैं।


In [21]:
def addDialogues():
    for i in range(70):
        text = dialogues.values[i][0]
        cblimg = cartoonizeblt_mem_nb('BorderedImage/' + str(i) + '.jpeg', 14, 1, 3, text, 1, 'English')
        cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)
        img = (Image.fromarray(cblimg_pil))
        img.save(f'images-comic/{i}.jpg')

In [22]:
def addDialogueshindi():
    for i in range(70):
        text = dialogues.values[i][1]
        cblimg = cartoonizeblt_mem_nb('BorderedImage/' + str(i) + '.jpeg', 14, 1, 3, text, 1, 'Hindi')
        cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)
        img = (Image.fromarray(cblimg_pil))
        img.save(f'images-comic-hindi/{i}.jpg')

In [23]:
def HorVerStitching():
    image_names=[f'images-comic/{i}.jpg' for i in range(0,70)]
    h_strip=image_names[0:70]
    k=0
    for i in range(7):
        sub_hstrip=[]
        sub_hstrip.append(h_strip[i*10:i*10+3]) #3 images
        sub_hstrip.append(h_strip[i*10+3:i*10+7]) #4 images
        sub_hstrip.append(h_strip[i*10+7:i*10+10]) # 3 images
        for j in range(3):
            imgs=[ pili.open(i) for i in sub_hstrip[j] ]
            min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
            imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
            imgs_comb = pili.fromarray(imgs_comb)
            imgs_comb.save(f'images-hstrips/{k}.jpg')
            k+=1
    #now performing verticalling stitching 
    strip_names = [f'images-hstrips/{i}.jpg' for i in range(k)]
    for i in range(7):
        imgs=[ pili.open(i) for i in strip_names[i*3:i*3+3]]
        min_shape = sorted([(np.sum(i.size),i.size) for i in imgs])[0][1]
        imgs_comb = np.vstack((np.asarray(i.resize(min_shape)) for i in imgs))
        imgs_comb = pili.fromarray( imgs_comb)
        imgs_comb.save(f'images-vstrips/{i}.jpg')

In [24]:
def HorVerStitchinghindi():
    image_names=[f'images-comic-hindi/{i}.jpg' for i in range(0,70)]
    h_strip=image_names[0:70]
    k=0
    for i in range(7):
        sub_hstrip=[]
        sub_hstrip.append(h_strip[i*10:i*10+3]) #3 images
        sub_hstrip.append(h_strip[i*10+3:i*10+7]) #4 images
        sub_hstrip.append(h_strip[i*10+7:i*10+10]) # 3 images
        for j in range(3):
            imgs=[ pili.open(i) for i in sub_hstrip[j] ]
            min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
            imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
            imgs_comb = pili.fromarray(imgs_comb)
            imgs_comb.save(f'images-hstrips-hindi/{k}.jpg')
            k+=1
    #now performing verticalling stitching 
    strip_names = [f'images-hstrips-hindi/{i}.jpg' for i in range(k)]
    for i in range(7):
        imgs=[ pili.open(i) for i in strip_names[i*3:i*3+3]]
        min_shape = sorted([(np.sum(i.size),i.size) for i in imgs])[0][1]
        imgs_comb = np.vstack((np.asarray(i.resize(min_shape)) for i in imgs))
        imgs_comb = pili.fromarray( imgs_comb)
        imgs_comb.save(f'images-vstrips-hindi/{i}.jpg')

In [30]:
def MakePDF():    
    cover = pili.open('images-vstrips/00.jpg')
    width, height = cover.size
    pdf = FPDF(unit = "pt", format = [width, height])
    pdf.add_page()
    pdf.image('images-vstrips/00.jpg', 0, 0, width, height)
    file=0
    
    image_list = [f'images-vstrips/{i}.jpg' for i in range(7)] #image_list is the list with all image filenames

    for image in image_list:
        pdf.add_page()
        pdf.image(image, 0, 0, width, height)
    pdf.output("ComicInEnglish.pdf", "F")
    print("Created comic in english- Enjoy reading the comic!")

In [33]:
def MakePDFhindi():    
    cover = pili.open('images-vstrips/00.jpg')
    width, height = cover.size
    pdf = FPDF(unit = "pt", format = [width, height])
    pdf.add_page()
    pdf.image('images-vstrips/00.jpg', 0, 0, width, height)
    file=0
    
    image_list = [f'images-vstrips-hindi/{i}.jpg' for i in range(7)] #image_list is the list with all image filenames

    for image in image_list:
        pdf.add_page()
        pdf.image(image, 0, 0, width, height)
    pdf.output("ComicInHindi.pdf", "F")
    print("Created comic in Hindi- Enjoy reading the comic!")

In [32]:
resize()
borderImages(borderedImgPath)
addDialogues()
HorVerStitching()
MakePDF()

/var/folders/vv/cbgwbmv57tlf1xzhsdbdw0lm0000gn/T/ipykernel_63769/2277992392.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
/var/folders/vv/cbgwbmv57tlf1xzhsdbdw0lm0000gn/T/ipykernel_63769/2277992392.py:22: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.vstack((np.asarray(i.resize(min_shape)) for i in imgs))


Created comic in english- Enjoy reading the comic!


In [34]:
resize()
borderImages(borderedImgPath)
addDialogueshindi()
HorVerStitchinghindi()
MakePDFhindi()

/var/folders/vv/cbgwbmv57tlf1xzhsdbdw0lm0000gn/T/ipykernel_63769/1364805212.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
/var/folders/vv/cbgwbmv57tlf1xzhsdbdw0lm0000gn/T/ipykernel_63769/1364805212.py:22: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.vstack((np.asarray(i.resize(min_shape)) for i in imgs))


Created comic in Hindi- Enjoy reading the comic!
